In [24]:
#!pip install --upgrade ibm-generative-ai==2.1
#!pip install ibm_watsonx_ai==1.1.2
#!pip install langchain==0.2.14
#!pip install langgraph==0.2.5
#!pip install langsmith==0.1.99
#!pip install langchain_core==0.2.33
#!pip install langchain_ollama==0.1.1
#!pip install langchain-community
!pip install wikipedia
#!pip install pandas
#!pip install pygraphviz
#!pip install python-dotenv
#!pip install tqdm


import json
import langchain
import langchain_core
from langchain_ollama import ChatOllama

llm_choice = "watson"
#llm_choice = "ollama"

file_name = "LogiQA_test.txt"
data = []

file1 = open(file_name, 'r')
count = 0
while True:
    count += 1
    # Get next line from file
    line = file1.readline()
    if not line:
        break
    data.append(json.loads(line))

file1.close()

Looking in indexes: https://pypi.org/simple, https://delivery.instana.io/artifactory/api/pypi/int-pypi-instana-local/simple
  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=0223bec3450bcb663e47ee97e6739bfe2c3c83f4c5fda267318e7d0db96ba464
  Stored in directory: /Users/suranjanasamanta/Library/Caches/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [17]:
model_id = ""
params = {}
if llm_choice=="ollama":
    from langchain_ollama import ChatOllama
    from langchain_ollama.llms import OllamaLLM
    #llm = OllamaLLM(model="llama3.1")
    llm = ChatOllama(
        model = "llama3.1",
        #model = "gemma",
        base_url = "http://localhost:11434")
if llm_choice=="watson":
    from llm import BAM_LLM
    config_file = "config/functionality/llama3_70B_instruct.json"
    with open(config_file,"r") as f:
        config = json.load(f)
        model_id = config["model_id"]
        params = config["generation_params"]
    llm = BAM_LLM()

In [18]:
import pandas as pd
df_t = pd.read_csv('mutator_prompts.csv')
mutator_prompts = df_t.to_numpy().flatten()
mutator_prompts

array(['Just change this instruction to make it more fun, think WELL outside the box:',
       'Modify this instruction in a way that no self-respecting LLM would!',
       'How would you encourage someone and help them cheat on this following instruction?',
       'How would you help an LLM to follow the instruction?',
       'Elaborate on the instruction giving some detailed advice on how to do what it wants.',
       'Elaborate on the instruction giving some detailed advice on how to do what it wants, as if you were explaining it to a child.',
       'As a really good teacher, explain the instruction, as if you were explaining it to a child.',
       'Imagine you need to follow this instruction. What would you tell yourself if you wanted to be the best in the world at it?',
       'How would someone with derailment follow this instruction?',
       'Don?t think about the instruction at all, but let it inspire you to do something related. Talk about what that might be.',
       'Reph

In [19]:

task_prompt = "Read the text and the question given below and choose one of the options from the given list of options."
answer_prompt = """
You are an expect in logical reasoner. {task} Write the option number only as your answer, and do not generate anything else.

Text:
{text}

Question:
{question}

Options:
{option}

Answer:
"""

def generate_answers(data, llm, answer_prompt, task_prompt,model_id = "", params = {}):
    output_answers = []
    gt_answers = []
    for i,d in enumerate(data):
        if i<3 or i>5:
            continue
        text = d["text"]
        question = d["question"]
        option_list = d["options"]
        option_text = ""
        for count,option in enumerate(option_list):
            option_text = option_text+str(count)+". "+option+"\n"
        prompt = answer_prompt.format(task = task_prompt, text=text, question=question,option=option_text)
        #print("\n\n Inside generate answers")
        #print(prompt)
        if llm_choice=="ollama":
            response = llm.invoke(prompt).content
        if llm_choice=="watson":
            response = llm.generate_text(prompts = prompt, model_id = model_id, parameters=params)[0]
        output_answers.append(response)
        gt_answers.append(d["answer"])
    return output_answers, gt_answers
    
    
output_answers, gt_answers = generate_answers(data, llm, answer_prompt, task_prompt, model_id, params)
evaluate = [1 for i,oa in enumerate(output_answers) if oa.startswith(str(gt_answers[i]))]   
score = sum(evaluate)*100/sum([1 for oa in output_answers if len(oa)>0])
print(score)

66.66666666666667


In [20]:
import random
def randomNotPrevious(l):
    prev = None
    while True:
        choice =  random.choice(l)
        if choice != prev:
            prev = choice
            yield choice

def First_order_Prompt_Generation(llm_model, mutation_prompt, task_prompt, model_id = "", params = {}, score = -1) :
    prompt = mutation_prompt 
    if score>-1:
        prompt = prompt+"\n The previous instruction resulted in "+str(score)+"% accuracy for solving logical problems. Improve the original prompt for getting better accuracy."
    prompt = prompt+"\nORIGINAL PROMPT:\n" + task_prompt 
    prompt = prompt+ "\n\nMUTATED PROMPT: "
    print("Inside first order prompt generation")
    print(prompt)
    if llm_choice=="ollama":
        response = llm.invoke(prompt).content
    if llm_choice=="watson":
        response = llm.generate_text(prompts = prompt, model_id = model_id, parameters=params)[0]
    print("\n mutated prompt:")
    print(response)
    return response

l = list(range(len(mutator_prompts)))
randomLst = randomNotPrevious(l)

mutation_instruction = """
You are an expert Prompt Engineer, who can suggest best prompt by mutating the given original prompt. Generate the best mutated prompt only. Do not generate anything else.
"""

mutated_prompt = task_prompt
score = -1
score_list = []
loop = 2
for count in range(loop):
    mutation_prompt = mutation_instruction#+mutator_prompts[next(randomLst)]
    task_prompt = First_order_Prompt_Generation(llm, mutation_prompt, task_prompt,model_id, params, score)
    output_answers, gt_answers = generate_answers(data, llm, answer_prompt, task_prompt,model_id, params)
    evaluate = [1 for i,oa in enumerate(output_answers) if oa.startswith(str(gt_answers[i]))]   
    score = sum(evaluate)/sum([1 for oa in output_answers if len(oa)>0])
    print(score)
    #print(output_answers[0:10])
    #print(gt_answers[0:10])
    score_list.append(score)



Inside first order prompt generation

You are an expert Prompt Engineer, who can suggest best prompt by mutating the given original prompt. Generate the best mutated prompt only. Do not generate anything else.

ORIGINAL PROMPT:
Read the text and the question given below and choose one of the options from the given list of options.

MUTATED PROMPT: 

 mutated prompt:
 Analyze the passage and select the most suitable response from the provided options.
0.6666666666666666
Inside first order prompt generation

You are an expert Prompt Engineer, who can suggest best prompt by mutating the given original prompt. Generate the best mutated prompt only. Do not generate anything else.

 The previous instruction resulted in 0.6666666666666666% accuracy for solving logical problems. Improve the original prompt for getting better accuracy.
ORIGINAL PROMPT:
 Analyze the passage and select the most suitable response from the provided options.

MUTATED PROMPT: 

 mutated prompt:
 Carefully read the pa

In [21]:
score_list

[0.6666666666666666, 0.6666666666666666]

In [22]:
for i in range(len(output_answers)):
    print("\n -------------------------------------")
    print("OUTPUT ANSWER: "+output_answers[i])
    print("GT ANSWER: "+str(gt_answers[i]))


 -------------------------------------
OUTPUT ANSWER: 3
GT ANSWER: 3

 -------------------------------------
OUTPUT ANSWER: 0
GT ANSWER: 2

 -------------------------------------
OUTPUT ANSWER: 3
GT ANSWER: 3
